In [1]:
import numpy as np  #Import NumPy library
import os
import networkx as nx
from tqdm import tqdm                                   # Import tqdm for progress bar

Inputs = [''] * 7
Inputs[0]= N = 1000    #Number_of_node
Inputs[1] = 0       #Initial_time
Inputs[2] = 0.1    #Time_step
Inputs[3] = 4      #Total_time
Inputs[4] = 1.5       #Start_coupling
Inputs[5] = 0.01     #coupling_step
Inputs[6] = 1.51       #End_coupling
Frustration=1.57    #Frustration
Matrix_I='Fully'    #adj matrix I [fully,Erdős_Rényi] (Intralayeradjacencymatrix)
Matrix_II='Fully'   #adj matrix II[fully,Erdős_Rényi] (Intralayeradjacencymatrix)

Interlayer_connection='Fully' #(Interlayer connection)
Intralayer_frustration=0       #b=(Intralayer frustration)
Interlayer_coupling=10          #L=Interlayer coupling

InitialPhases='Uniform'

Natfreq_min=-0.5
Natfreq_max=0.5

Natfreq_dw=0.8



# Data
cleaned_inputs = [str(int(x)) if isinstance(x, (int, float)) and x == int(x) else str(x) for x in Inputs]
np.savetxt('./data.txt', cleaned_inputs, fmt='%s')


# Example
os.makedirs('./Example', exist_ok=True)
# Frustration
os.makedirs('./Example/a=Interlayer frustration', exist_ok=True)
Frustration_array = np.full(N, Frustration)
np.savetxt('./Example/a=Interlayer frustration/Layer1to2.txt', Frustration_array, fmt='%s')
np.savetxt('./Example/a=Interlayer frustration/Layer2to1.txt', Frustration_array, fmt='%s')

# Matrix
# Intralayer adjacency matrices
os.makedirs('./Example/A=Intralayeradjacencymatrix', exist_ok=True)
if Matrix_I=='Fully':
    adj_I = np.ones((N, N))
    np.fill_diagonal(adj_I, 0)
    np.savetxt('./Example/A=Intralayeradjacencymatrix/Layer1.txt', adj_I, fmt='%i')
if Matrix_II=='Fully':
    adj_II = np.ones((N, N))
    np.fill_diagonal(adj_II, 0)
    np.savetxt('./Example/A=Intralayeradjacencymatrix/Layer2.txt', adj_II, fmt='%i')
if Matrix_I=='Erdős_Rényi':
    adj_I = nx.to_numpy_array(nx.erdos_renyi_graph(n=N, p=1))
    np.savetxt('./Example/A=Intralayeradjacencymatrix/Layer1.txt', adj_I, fmt='%f')
if Matrix_II=='Erdős_Rényi':
    adj_II = nx.to_numpy_array(nx.erdos_renyi_graph(n=N, p=1))
    np.savetxt('./Example/A=Intralayeradjacencymatrix/Layer2.txt', adj_II, fmt='%f')


#Interlayer connection
os.makedirs('./Example/B=Interlayer connection', exist_ok=True)
if Interlayer_connection=='Fully':
    arr_connection=np.ones(N)
    np.savetxt('./Example/B=Interlayer connection/Layer1to2.txt', arr_connection, fmt='%i')
    np.savetxt('./Example/B=Interlayer connection/Layer2to1.txt', arr_connection, fmt='%i')

#Intralayer frustration
os.makedirs('./Example/b=Intralayer frustration', exist_ok=True)
Intralayer_frustration_array = np.full((N,N), Intralayer_frustration)
np.savetxt('./Example/b=Intralayer frustration/Layer1.txt', Intralayer_frustration_array, fmt='%i', delimiter='\t')
np.savetxt('./Example/b=Intralayer frustration/Layer2.txt', Intralayer_frustration_array, fmt='%i', delimiter='\t')

#I=InitialPhases
os.makedirs('./Example/I=InitialPhases', exist_ok=True)
if InitialPhases=='Uniform':
    initial_theta_I = np.random.uniform(-np.pi, np.pi, size=N)
    initial_theta_II = np.random.uniform(-np.pi, np.pi, size=N)
    np.savetxt('./Example/I=InitialPhases/origin1.txt', initial_theta_I, fmt='%.3f')
    np.savetxt('./Example/I=InitialPhases/origin2.txt', initial_theta_II, fmt='%.3f')


#L=Interlayer_coupling
os.makedirs('./Example/L=Interlayer coupling', exist_ok=True)
Interlayer_coupling_array = np.full(N, Interlayer_coupling)
np.savetxt('./Example/L=Interlayer coupling/Layer1to2.txt', Interlayer_coupling_array, fmt='%i')
np.savetxt('./Example/L=Interlayer coupling/Layer2to1.txt', Interlayer_coupling_array, fmt='%i')


#W=Naturalfrequency
os.makedirs('./Example/W=Naturalfrequency', exist_ok=True)
natfreqs_I = np.linspace(Natfreq_min, Natfreq_max, N)                      # Create linearly spaced array x_I from -0.5 to 0.5 with 1000 points
natfreqs_II = np.linspace(Natfreq_min, Natfreq_max, N)                     # Create another identical array x_II
def calculate_dw(x_I, x_II):                            # Function to calculate dw based on absolute differences
    N = len(x_I)                                        # Number of points
    total_abs_diff = np.sum(np.abs(x_I - x_II))         # Sum of absolute node-wise differences
    dw = total_abs_diff / (2 * np.sum(np.abs(x_I)))     # Compute dw using the given formula
    return dw
def adjust_nodes(x_I, x_II, target_dw=Natfreq_dw):             # Function to adjust nodes in x_II to increase dw (with progress bar)
    max_iters = len(x_II) // 2                          # Max possible swaps (symmetric)
    with tqdm(total=max_iters, desc="Adjusting nodes", unit="swap") as pbar:
        for iteration in range(max_iters):
            dw = calculate_dw(x_I, x_II)                # Compute current dw
            if dw >= target_dw:                         # Stop if target met
                break                                   # Swap mirrored nodes in x_II
            x_II[iteration], x_II[-iteration - 1] = x_II[-iteration - 1], x_II[iteration]
            pbar.update(1)                              # Update progress bar
    return x_II,dw
natfreqs_II,final_dw = adjust_nodes(natfreqs_I, natfreqs_II)                 # Adjust x_II nodes to meet target dw
print(f"Final dw: {final_dw}")
np.savetxt('./Example/W=Naturalfrequency/Layer1.txt', natfreqs_I, fmt='%.5f')
np.savetxt('./Example/W=Naturalfrequency/Layer2.txt', natfreqs_II, fmt='%.5f')




#create save folders
os.makedirs('./Save', exist_ok=True)
os.makedirs('./Save/Average_Syncrony(couplig_SyncL1_SyncL2_Extime)', exist_ok=True)
os.makedirs('./Save/Group_synch', exist_ok=True)
os.makedirs('./Save/Last_Phase', exist_ok=True)
os.makedirs('./Save/Phases(time)VS(Node)', exist_ok=True)
os.makedirs('./Save/Syncrony(time_SyncL1_SyncL2)', exist_ok=True)



Adjusting nodes:  55%|█████▌    | 277/500 [00:00<?, ?swap/s]

Final dw: 0.8010840000000001


# Kuramoto.Version5.h

In [2]:
cpp_code = """#ifndef KURAMOTO_VERSION5_H_INCLUDED
#define KURAMOTO_VERSION5_H_INCLUDED
/*****************************************************************************************************************************/
/*** In our simulation, we consider N = 1000. The initial phases of the oscillators are randomly sampled from a uniform    ***/
/*** distribution within the range -pi to pi. To obtain the results, we numerically solve the equations described   ***/
/*** in Equation (1) using the fourth-order Runge-Kutta method with a time step of dt = 0.01. The simulation is conducted  ***/
/*** for a total of 40,000 steps. In our simulation, we calculate the average RI(II) over the final 80p of the simulation  ***/
/*** duration, which corresponds to the period when the system has settled into a steady state.                            ***/
/*****************************************************************************************************************************/
/*** Topic: Dynamic Runge-Kutta 4th Order Method application								                               ***/
/***        solved numerically using RungeKutta 4th order method                                                           ***/
/*** Investigating the effect of frequency arrangement of the nodes in the dynamics of two-layer networks                  ***/
/*** Version Release 17.12 rev 11256                                                Ali-Seif                               ***/
/*** Github address:                                            https://github.com/AliSeif96                               ***/
/***                                                            https://github.com/Articles-data/Frequency-Arrangement     ***/
/*** The latest code update: 09/17/2024                                                                                    ***/
/*** Code implemented in Code:Visual Studio Code V 1.93.1                                                                  ***/
/*** MSI: PX60 6QD/ DDR4                                                                                                   ***/
/*** Run under a Intel Core i7-6700HQ CPU @ 2.60GHz  64 based processor with 16 GB RAM                                     ***/
/*****************************************************************************************************************************/
//$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
#include<iostream>//for cout                                                                                               $$$$
#include<fstream>//infile /ofstream                                                                                        $$$$
#include <string>//for stod( )                                                                                             $$$$
#include <sstream>//stringstream ss(line)                                                                                  $$$$
#include<ctime>//For Example clock()                                                                                       $$$$
#include <cmath>//For Example pow                                                                                          $$$$
#include <omp.h>//                                                                                                         $$$$
#include <stdio.h>//                                                                                                       $$$$
#define Pi 3.141592653589793238462643383279502884//pi number                                                               $$$$
using namespace std;//                                                                                                     $$$$
//$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
//-----------------------------------------------------------------------------------------------------------------------------
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                          Runge-Kutta 4th                                                $$$$
//                                                          --------------                                                 $$$$
//                                                          \            /                                                 $$$$
//                                                           \          /                                                  $$$$
//                                                            \        /                                                   $$$$
//                                                             \      /                                                    $$$$
//                                                              \    /                                                     $$$$
//                                                               \  /                                                      $$$$
//                                                                \/                                                       $$$$
//-----------------------------------------------------------------------------------------------------------------------------
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                                     dydt                                       @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double dydt(int specified,int N,double coupling,double W,double L,double B,         //@@@                                   ---
            double a,const double* b,const double* A,double Phase_old_otherlayer,   //@@@                                   ---
            double* Phase_old,double Phase_old_specified)                           //@@@                                   ---
{                                                                                   //@@@                                   ---
    double summation = 0.0;                                                         //@@@                                   ---
    for (int i = 0; i < N; i++){                                                    //@@@                                   ---
        summation += (A[i] * sin((Phase_old[i] - Phase_old_specified + b[i])));     //@@@                                   ---
    }                                                                               //@@@                                   ---
    double connection = 0.0;                                                        //@@@                                   ---
    connection = (L*B*sin(Phase_old_otherlayer-Phase_old_specified+a));             //@@@                                   ---
    double k = 0;                                                                   //@@@      connection calculated        ---
    k = W + ((coupling/(N * 1.0))*summation) + connection;                          //@@@               all sum             ---
    return k;                                                                       //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                                CCRK4                                           @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
void Runge_Kutta_4(int N,double dt,double coupling,const double* W,const double* L, //@@@                                   ---
                   const double* B,const double* a,const double* const* b,          //@@@                                   ---
                   const double* const* A,double* Phase_old,                        //@@@                                   ---
                   double* Phase_old_otherlayer,double* Phase_new)                  //@@@                                   ---
{                                                                                   //@@@                                   ---
    for (int i = 0; i < N; i++)                                                     //@@@                                   ---
        {                                                                           //@@@                                   ---   
            double k1 = dydt(i,N,coupling,W[i],L[i],B[i],a[i],b[i],A[i],            //@@@                                   ---
                            Phase_old_otherlayer[i],Phase_old,Phase_old[i]);        //@@@                                   ---                                                                       //@@@                                   ---
            double k2 = dydt(i,N,coupling,W[i],L[i],B[i],a[i],b[i],A[i],            //@@@                                   ---
                            Phase_old_otherlayer[i],Phase_old,Phase_old[i]+k1*dt/2.0);//@                                   ---
            double k3 = dydt(i,N,coupling,W[i],L[i],B[i],a[i],b[i],A[i],            //@@@                                   ---
                            Phase_old_otherlayer[i],Phase_old,Phase_old[i]+k2*dt/2.0);//@                                   ---
            double k4 = dydt(i,N,coupling,W[i],L[i],B[i],a[i],b[i],A[i],            //@@@                                   ---
                            Phase_old_otherlayer[i],Phase_old,Phase_old[i]+k3*dt);  //@@@                                   ---
            Phase_new[i] = Phase_old[i]+dt/6.0*(k1+2.0*k2+2.0*k3+k4);               //@@@                                   ---
        }                                                                           //@@@                                   ---
}                                                                                   //@@@                                   ---
//-----------------------------------------------------------------------------------------------------------------------------
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                         Order Parameter                                                 $$$$
//                                                          --------------                                                 $$$$
//                                                          \            /                                                 $$$$
//                                                           \          /                                                  $$$$
//                                                            \        /                                                   $$$$
//                                                             \      /                                                    $$$$
//                                                              \    /                                                     $$$$
//                                                               \  /                                                      $$$$
//                                                                \/                                                       $$$$
//-----------------------------------------------------------------------------------------------------------------------------
//-----------------------------------------------------------------------------------------------------------------------------
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                                Check scale -pi tp pi                           @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
void check_scale(int N, double* phi)                                                //@@@                                   ---
{                                                                                   //@@@                                   ---
    for (int i = 0; i < N; i++)                                                     //@@@                                   ---
    {                                                                               //@@@                                   ---
        while(abs(phi[i])>Pi){                                                      //@@@                                   ---
            if (phi[i]>0){                                                          //@@@                                   ---
                phi[i]=phi[i]-2*Pi;                                                 //@@@                                   ---
            }else if(phi[i]<0){                                                     //@@@                                   ---
                phi[i]=phi[i]+2*Pi;                                                 //@@@                                   ---
            }                                                                       //@@@                                   ---
        }                                                                           //@@@                                   ---
    }                                                                               //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                                order_parameter                                 @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double order_parameter(int N, double* phi)                                          //@@@                                   ---
{                                                                                   //@@@                                   ---
    double rc = 0.0, rs = 0.0;                                                      //@@@                                   ---
    for (int j = 0; j < N; j++)                                                     //@@@                                   ---
    {                                                                               //@@@                                   ---
        rc += cos(phi[j]);                                                          //@@@                                   ---
        rs += sin(phi[j]);                                                          //@@@                                   ---
    }                                                                               //@@@                                   ---
    return sqrt(pow(rc, 2) + pow(rs, 2)) / (1.0 * N);                               //@@@                                   ---
}                                                                                   //@@@                                   ---











double TGroup_order_parameter(double* phi)                                          //@@@                                   ---
{                                                                                   //@@@                                   ---
    int First=0;
    int End=1000;   
    int size=End-First;

    double rc = 0.0, rs = 0.0;                                                      //@@@                                   ---
    for (int j = 0; j < 1000; j++)                                                     //@@@                                   ---
    {                                                                               //@@@                                   ---
        rc += cos(phi[j]);                                                          //@@@                                   ---
        rs += sin(phi[j]);                                                          //@@@                                   ---
    }                                                                               //@@@                                   ---
    return sqrt(pow(rc, 2) + pow(rs, 2)) / (1.0 * size);                               //@@@                                   ---
}                                                                                   //@@@                                   ---
double LGroup_order_parameter(double* phi)                                          //@@@                                   ---
{                                                                                   //@@@                                   ---
    int First=0;
    int End=277;
    int size=End-First;
 
    double rc = 0.0, rs = 0.0;                                                      //@@@                                   ---
    for (int j = 0; j < 277; j++)                                                     //@@@                                   ---
    {                                                                               //@@@                                   ---
        rc += cos(phi[j]);                                                          //@@@                                   ---
        rs += sin(phi[j]);                                                          //@@@                                   ---
    }                                                                               //@@@                                   ---
    return sqrt(pow(rc, 2) + pow(rs, 2)) / (1.0 * size);                               //@@@                                   ---
}                                                                                   //@@@                                   ---
double MGroup_order_parameter( double* phi)                                          //@@@                                   ---
{                                                                                   //@@@                                   ---
    int First=277;
    int End=723;  
    int size=End-First;

    double rc = 0.0, rs = 0.0;                                                      //@@@                                   ---
    for (int j = 277; j < 723; j++)                                                     //@@@                                   ---
    {                                                                               //@@@                                   ---
        rc += cos(phi[j]);                                                          //@@@                                   ---
        rs += sin(phi[j]);                                                          //@@@                                   ---
    }                                                                               //@@@                                   ---
    return sqrt(pow(rc, 2) + pow(rs, 2)) / (1.0 * size);                               //@@@                                   ---
}                                                                                   //@@@                                   ---
double RGroup_order_parameter( double* phi)                                          //@@@                                   ---
{      
    int First=723;
    int End=1000;                                                                             //@@@                                   ---
    int size=End-First;

    double rc = 0.0, rs = 0.0;                                                      //@@@                                   ---
    for (int j = First; j < End; j++)                                                     //@@@                                   ---
    {                                                                               //@@@                                   ---
        rc += cos(phi[j]);                                                          //@@@                                   ---
        rs += sin(phi[j]);                                                          //@@@                                   ---
    }                                                                               //@@@                                   ---
    return sqrt(pow(rc, 2) + pow(rs, 2)) / (1.0 * size);                               //@@@                                   ---
}                                                                                   //@@@                                   ---








//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                                  previous phases                               @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double* for_loop_equal(double* Phase) {                                             //@@@calculate initial theta            ---
    return Phase;                                                                   //@@@                                   ---
}                                                                                   //@@@                                   ---
//-----------------------------------------------------------------------------------------------------------------------------
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                        read file to arrey                                               $$$$
//                                                          --------------                                                 $$$$
//                                                          \            /                                                 $$$$
//                                                           \          /                                                  $$$$
//                                                            \        /                                                   $$$$
//                                                             \      /                                                    $$$$
//                                                              \    /                                                     $$$$
//                                                               \  /                                                      $$$$
//                                                                \/                                                       $$$$
//-----------------------------------------------------------------------------------------------------------------------------
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                               W=Naturalfrequency .txt                          @@@@ Read data from text               ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double* read_1D_W(string Filename, int Numberofnode)                                //@@@ (Phases & frequency & Matrix)     ---
{                                                                                   //@@@                                   ---
    double* data_1D = new double[Numberofnode];                                     //@@@                                   ---
    ifstream file("./Example/W=Naturalfrequency/" + Filename + ".txt");             //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING! W=Natural frequency file is not here!" << endl;           //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
        for (int i = 0; i < Numberofnode; i++)                                      //@@@                                   ---
        {                                                                           //@@@                                   ---
            file >> data_1D[i];                                                     //@@@                                   ---
        }                                                                           //@@@                                   ---
        cout << "W of "<<Filename + "\\tloaded\\t First data=" <<                     //@@@                                   ---
         data_1D[0]<< "\\tLastst data="<< data_1D[Numberofnode-1] <<endl;            //@@@                                   ---
    }                                                                               //@@@                                   ---
    file.close();                                                                   //@@@                                   ---
    return data_1D;                                                                 //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                               I=InitialPhases .txt                             @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double* read_1D_I(string Filename, int Numberofnode)                                //@@@                                   ---
{                                                                                   //@@@                                   ---
    double* data_1D = new double[Numberofnode];                                     //@@@                                   ---
    ifstream file("./Example/I=InitialPhases/" + Filename + ".txt");                //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING! W=Natural frequency file is not here!" << endl;           //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
        for (int i = 0; i < Numberofnode; i++)                                      //@@@                                   ---
        {                                                                           //@@@                                   ---
            file >> data_1D[i];                                                     //@@@                                   ---
        }                                                                           //@@@                                   ---
        cout << "I of "<<Filename + "\\tloaded\\t First data=" <<                     //@@@                                   ---
         data_1D[0]<< "\\tLastst data="<< data_1D[Numberofnode-1] <<endl;            //@@@                                   ---
    }                                                                               //@@@                                   ---
    file.close();                                                                   //@@@                                   ---
    return data_1D;                                                                 //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                       B=Interlayer connection .txt                             @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double* read_1D_B(string Filename, int Numberofnode)                                //@@@                                   ---
{                                                                                   //@@@                                   ---
    double* data_1D = new double[Numberofnode];                                     //@@@                                   ---
    ifstream file("./Example/B=Interlayer connection/" + Filename + ".txt");        //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING!\\tB=Interlayer connection\\t"<<Filename<<                   //@@@                                   ---
        " file is not here!" << endl;                                               //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
        for (int i = 0; i < Numberofnode; i++)                                      //@@@                                   ---
        {                                                                           //@@@                                   ---
            file >> data_1D[i];                                                     //@@@                                   ---
        }                                                                           //@@@                                   ---
        cout << "B of "<<Filename + "\\tloaded\\t First data=" <<                     //@@@                                   ---
         data_1D[0]<< "\\tLastst data="<< data_1D[Numberofnode-1] <<endl;            //@@@                                   ---
    }                                                                               //@@@                                   ---
    file.close();                                                                   //@@@                                   ---
    return data_1D;                                                                 //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                       B=Interlayer connection .txt                             @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double* read_1D_a(string Filename, int Numberofnode)                                //@@@                                   ---
{                                                                                   //@@@                                   ---
    double* data_1D = new double[Numberofnode];                                     //@@@                                   ---
    ifstream file("./Example/a=Interlayer frustration/" + Filename + ".txt");       //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING!\\ta=Interlayer frustration\\t"<<Filename<<                  //@@@                                   ---
        " file is not here!" << endl;                                               //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
        for (int i = 0; i < Numberofnode; i++)                                      //@@@                                   ---
        {                                                                           //@@@                                   ---
            file >> data_1D[i];                                                     //@@@                                   ---
        }                                                                           //@@@                                   ---
        cout << "a of "<<Filename + "\\tloaded\\t First data=" <<                     //@@@                                   ---
         data_1D[0]<< "\\tLastst data="<< data_1D[Numberofnode-1] <<endl;            //@@@                                   ---
    }                                                                               //@@@                                   ---
    file.close();                                                                   //@@@                                   ---
    return data_1D;                                                                 //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                       B=Interlayer connection .txt                             @@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double* read_1D_L(string Filename, int Numberofnode)                                //@@@                                   ---
{                                                                                   //@@@                                   ---
    double* data_1D = new double[Numberofnode];                                     //@@@                                   ---
    ifstream file("./Example/L=Interlayer coupling/" + Filename + ".txt");          //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING!\\tL=Interlayer coupling\\t"<<Filename<<                     //@@@                                   ---
        " file is not here!" << endl;                                               //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
        for (int i = 0; i < Numberofnode; i++)                                      //@@@                                   ---
        {                                                                           //@@@                                   ---
            file >> data_1D[i];                                                     //@@@                                   ---
        }                                                                           //@@@                                   ---
        cout << "L of "<<Filename + "\\tloaded\\t First data=" <<                     //@@@                                   ---
         data_1D[0]<< "\\tLastst data="<< data_1D[Numberofnode-1] <<endl;            //@@@                                   ---
    }                                                                               //@@@                                   ---
    file.close();                                                                   //@@@                                   ---
    return data_1D;                                                                 //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                          Read matrix connection                               //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double** read_2D_b(string Filename, int Numberofnode)                               //@@@                                   ---
{                                                                                   //@@@                                   ---
    double** data_2D = new double* [Numberofnode];                                  //@@@                                   ---
    for (int i = 0; i < Numberofnode; i++)                                          //@@@                                   ---
        data_2D[i] = new double[Numberofnode];                                      //@@@                                   ---
    ifstream file("./Example/b=Intralayer frustration/" + Filename + ".txt");       //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING!\\tb=Intralayer frustration matrix\\t"<<Filename<<           //@@@                                   ---
        " file is not here!" << endl;                                               //@@@                                   ---
        return data_2D;                                                             //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
        for (int i = 0; i < Numberofnode; i++)                                      //@@@                                   ---
        {                                                                           //@@@                                   ---
            for (int j = 0; j < Numberofnode; j++)                                  //@@@                                   ---
            {                                                                       //@@@                                   ---
                double elem = 0;                                                    //@@@                                   ---
                file >> elem;                                                       //@@@                                   ---
                data_2D[i][j] = elem;                                               //@@@                                   ---
            }                                                                       //@@@                                   ---
        }                                                                           //@@@                                   ---
    }                                                                               //@@@                                   ---
            cout << "b of "<<Filename + "\\tloaded\\t First data=" <<                 //@@@                                   ---
         data_2D[0][1]<< "\\tLastst data="<< data_2D[0][Numberofnode-1] <<endl;      //@@@                                   ---
    return data_2D;                                                                 //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                          Read matrix connection                               //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double** read_2D_A(string Filename, int Numberofnode)                               //@@@                                   ---
{                                                                                   //@@@                                   ---
    double** data_2D = new double* [Numberofnode];                                  //@@@                                   ---
    for (int i = 0; i < Numberofnode; i++)                                          //@@@                                   ---
        data_2D[i] = new double[Numberofnode];                                      //@@@                                   ---
    ifstream file("./Example/A=Intralayeradjacencymatrix/" + Filename + ".txt");    //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING!\\tA=Intralayer adjacency matrix\\t"<<Filename<<             //@@@                                   ---
        " file is not here!" << endl;                                               //@@@                                   ---
        return data_2D;                                                             //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
        for (int i = 0; i < Numberofnode; i++)                                      //@@@                                   ---
        {                                                                           //@@@                                   ---
            for (int j = 0; j < Numberofnode; j++)                                  //@@@                                   ---
            {                                                                       //@@@                                   ---
                double elem = 0;                                                    //@@@                                   ---
                file >> elem;                                                       //@@@                                   ---
                data_2D[i][j] = elem;                                               //@@@                                   ---
            }                                                                       //@@@                                   ---
        }                                                                           //@@@                                   ---
    }                                                                               //@@@                                   ---
            cout << "A of "<<Filename + "\\tloaded\\t First data=" <<                 //@@@                                   ---
         data_2D[0][1]<< "\\tLastst data="<< data_2D[0][Numberofnode-1] <<endl;      //@@@                                   ---
    return data_2D;                                                                 //@@@                                   ---
}                                                                                   //@@@                                   ---
//-----------------------------------------------------------------------------------------------------------------------------
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                             data.txt                                                    $$$$
//                                                          --------------                                                 $$$$
//                                                          \            /                                                 $$$$
//                                                           \          /                                                  $$$$
//                                                            \        /                                                   $$$$
//                                                             \      /                                                    $$$$
//                                                              \    /                                                     $$$$
//                                                               \  /                                                      $$$$
//                                                                \/                                                       $$$$
//-----------------------------------------------------------------------------------------------------------------------------
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                               count rows file in .txt                         //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
int count_rows_file(string file1)                                                   //@@@                                   ---
{                                                                                   //@@@                                   ---
    int rows = 0, cols = 0;                                                         //@@@                                   ---
    string line, item;                                                              //@@@                                   ---
    ifstream file(file1);                                                           //@@@                                   ---
    if (!file)                                                                      //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING! Data file is not here!" << endl;                          //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   --- 
        while (getline(file, line))                                                 //@@@                                   ---
        {                                                                           //@@@                                   ---
            rows++;                                                                 //@@@                                   ---
            if (rows == 1)                                                          //@@@First row only:                    ---
            {                                                                       //@@@determine the number of columns    ---
                stringstream ss(line);                                              //@@@Set up up a stream from this line  ---
                while (ss >> item) cols++;                                          //@@@Each item delineated by spaces     ---
            }                                                                       //@@@adds one to cols                   ---
        }                                                                           //@@@                                   ---
        file.close();                                                               //@@@                                   ---
        cout << "\\nFile had " << rows << " rows" << endl;                           //@@@                                   ---
    }                                                                               //@@@                                   ---
    return rows;                                                                    //@@@                                   ---
}                                                                                   //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
//@@@                               Read Data in .txt                               //@@@                                   ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                   ---
double* read_data(string name_of_file)                                              //@@@                                   ---
{                                                                                   //@@@                                   ---
    int rows=count_rows_file(name_of_file);                                         //@@@                                   ---
    double* data = new double[rows+1];                                              //@@@                                   ---
    string kk;                                                                      //@@@                                   ---
    ifstream fp(name_of_file);                                                      //@@@                                   ---
    if (!fp)                                                                        //@@@                                   ---
    {                                                                               //@@@                                   ---
        cout << "WARNING! Data file is not here!" << endl;                          //@@@                                   ---
    }                                                                               //@@@                                   ---
    else                                                                            //@@@                                   ---
    {                                                                               //@@@                                   ---
       string line, item;                                                           //@@@                                   ---
        for (int i=0;i<rows;i++){                                                   //@@@                                   ---
             fp >> kk;                                                              //@@@                                   ---
            data[i+1] = stod(kk);                                                   //@@@                                   ---
        }                                                                           //@@@                                   ---
        for (int x=1;x<=rows;x++){                                                  //@@@                                   ---
            cout <<"data["<<x<<"]=\\t" <<data[x] << endl;                            //@@@                                   ---
        }                                                                           //@@@                                   ---
    }                                                                               //@@@                                   ---
    fp.close();                                                                     //@@@                                   ---
    return data;                                                                    //@@@                                   ---
}                                                                                   //@@@                                   ---
//-----------------------------------------------------------------------------------------------------------------------------
#endif // KURAMOTO_VERSION5_H_INCLUDED

"""
# ذخیره در فایل
with open('./Kuramoto.Version5.h', 'w') as f:
    f.write(cpp_code)

print("✅  .با موفقیت ساخته شد Kuramoto.Version5.h فایل ")


✅  .با موفقیت ساخته شد Kuramoto.Version5.h فایل 


In [3]:
cpp_code = """/*****************************************************************************************************************************/
/*** In our simulation, we consider N = 1000. The initial phases of the oscillators are randomly sampled from a uniform    ***/
/*** distribution within the range -pi to pi. To obtain the results, we numerically solve the equations described   ***/
/*** in Equation (1) using the fourth-order Runge-Kutta method with a time step of dt = 0.01. The simulation is conducted  ***/
/*** for a total of 40,000 steps. In our simulation, we calculate the average RI(II) over the final 80p of the simulation  ***/
/*** duration, which corresponds to the period when the system has settled into a steady state.                            ***/
/*****************************************************************************************************************************/
/*** Topic: Dynamic Runge-Kutta 4th Order Method application								                               ***/
/***        solved numerically using RungeKutta 4th order method                                                           ***/
/*** Investigating the effect of frequency arrangement of the nodes in the dynamics of two-layer networks                  ***/
/*** Version Release 17.12 rev 11256                                                Ali-Seif                               ***/
/*** Github address:                                            https://github.com/AliSeif96                               ***/
/***                                                            https://github.com/Articles-data/Frequency-Arrangement     ***/
/*** The latest code update: 09/17/2024                                                                                    ***/
/*** Code implemented in Code:Visual Studio Code V 1.93.1                                                                  ***/
/*** MSI: PX60 6QD/ DDR4                                                                                                   ***/
/*** Run under a Intel Core i7-6700HQ CPU @ 2.60GHz  64 based processor with 16 GB RAM                                     ***/
/*****************************************************************************************************************************/
//$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
#include"Kuramoto.Version5.h"//import Internal library Kuramoto                                                            $$$$
#include <time.h>//import External library for calculate time                                                              $$$$
#include <iomanip>//                                                                                                       $$$$
//-------------------------------------------------------------------------------------------------------------------------$$$$
//                                                              |    |                                                     $$$$
//                                                              |    |                                                     $$$$
//                                                               main                                                      $$$$
//                                                          --------------                                                 $$$$
//                                                          \            /                                                 $$$$
//                                                           \          /                                                  $$$$
//                                                            \        /                                                   $$$$
//                                                             \      /                                                    $$$$
//                                                              \    /                                                     $$$$
//                                                               \  /                                                      $$$$
//                                                                \/                                                       $$$$
//                                           T1=W1+k/N*sum(A1*sin(I1+b1))+L12*B1*sin(I2+a1)                                $$$$
//-----------------------------------------------------------------------------------------------------------------------------
int main(){                                                                     //@@@           Beginning main              ---
    //-------------------------------------------------------------------------------------------------------------------------
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    Read and definition data,          ---
    //@@@                                     data.txt and Example file          @@@@    Number_of_node,Phases_initial,     ---
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    frequency,adj,coupling,delay,time  ---
    const double* data=read_data("data.txt");                                   //@@@ read data from data.txt and write them---
    const int Number_of_node = int(data[1]);                                    //@@@        N=Number_of_node=1000          ---
    cout << "|------------------------------------------------------|\\n"<< endl;//@@@                                       ---
    const double* frequency_layer1 = read_1D_W("Layer1",Number_of_node);     //@@@        w=natural frequency      L1    ---
    const double* frequency_layer2 = read_1D_W("Layer2",Number_of_node);     //@@@        w=natural frequency      L2    ---
    double* Phases_initial_layer1 = read_1D_I("origin1",Number_of_node);        //@@@        I=initial Phases         L1    ---
    double* Phases_initial_layer2 = read_1D_I("origin2",Number_of_node);        //@@@        I=initial Phases         L2    ---
    const double* const* adj_layer1 = read_2D_A("Layer1",Number_of_node);       //@@@        A=adjacency matrix       L1    ---
    const double* const* adj_layer2 = read_2D_A("Layer2",Number_of_node);       //@@@        A=adjacency matrix       L2    ---
    const double* bdj_layer1 = read_1D_B("Layer1to2",Number_of_node);           //@@@        B=Interlayer connection  L1    ---
    const double* bdj_layer2 = read_1D_B("Layer2to1",Number_of_node);           //@@@        B=Interlayer connection  L2    ---
    const double* frust_layer1 = read_1D_a("Layer1to2",Number_of_node);         //@@@        a=Interlayer frustration L1    ---
    const double* frust_layer2 = read_1D_a("Layer2to1",Number_of_node);         //@@@        a=Interlayer frustration L2    ---
    const double* const* Intrafrust_layer1 = read_2D_b("Layer1",Number_of_node);//@@@        b=Intralayer frustration L1    ---
    const double* const* Intrafrust_layer2 = read_2D_b("Layer2",Number_of_node);//@@@        b=Intralayer frustration L2    ---
    const double* inter_layer1 = read_1D_L("Layer1to2",Number_of_node);         //@@@        L=Interlayer coupling    L1    ---
    const double* inter_layer2 = read_1D_L("Layer2to1",Number_of_node);         //@@@        L=Interlayer coupling    L2    ---
    cout << "|------------------------------------------------------|\\n"<< endl;//@@@                                       ---
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                       ---
    //@@@                                definitions                             @@@@                                       ---
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                       ---
    const int time_stationary = int(data[4] * 0.2);                             //@@@    example T=20 time_stationary= 10   ---
    const int Number_Steps_time_stationary = int(time_stationary / data[3]);    //@@@   for example T=20 dt=0.01 >> = 1000  ---
    int coupling_step = round(data[5]/data[6]);                                 //@@@                                       ---
    ofstream Average_Syncrony(                                                  //@@@                                       ---
        "Save/Average_Syncrony(couplig_SyncL1_SyncL2_Extime)/output.txt");      //@@@       Create Sync file                ---
    ofstream Group_Syncrony(                                                  //@@@                                       ---
        "Save/Group_synch/output.txt");      //@@@       Create Sync file                ---
        double* Phases_next_layer1 = new double[Number_of_node];                    //@@@    Definition Phases next             ---
    double* Phases_next_layer2 = new double[Number_of_node];                    //@@@                                       ---
    double* Phases_layer1_previous = for_loop_equal(Phases_initial_layer1);     //@@@               Phases changer          ---
    double* Phases_layer2_previous = for_loop_equal(Phases_initial_layer2);     //@@@               [node][delay]           ---
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    //@@@                                      coupling loop                    //@@@                                       ---@
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                       ---@
    for (coupling_step;coupling_step <= int(data[7]/data[6]);coupling_step++){  //@@@                                       ---@
        double coupling=coupling_step*data[6];                                  //@@@            call coupling              ---@
        time_t start = time(NULL);                                              //@@@         reset time to zero            ---@
        ostringstream ostrcoupling;                                             //@@@    declaring output string stream     ---@
        ostrcoupling << fixed << setprecision(2) << coupling;                   //@@@  Sending a number as a stream output  ---@
        string strcoupling = ostrcoupling.str();                                //@@@ the str() converts number into string ---@
        //ofstream Phases_layer2("Save/Phases(time)VS(Node)/L2_k="+               //@@@       create file for phases L2       ---@
        //                       strcoupling+"layer2.txt");                       //@@@                                       ---@
        //ofstream Phases_layer1("Save/Phases(time)VS(Node)/L1_k="+               //@@@       create file for phases L1       ---@
         //                      strcoupling+"layer1.txt");                       //@@@                                       ---@
        double Total_syncrony_layer1 = 0;                                       //@@@          def tot synchroney L1        ---@
        double Total_syncrony_layer2 = 0;                                       //@@@          def tot synchroney L2        ---@
        
        double T_syncrony_layer1 = 0;                                       //@@@          def tot synchroney L1        ---@
        double L_syncrony_layer1 = 0;                                       //@@@          def tot synchroney L1        ---@
        double M_syncrony_layer1 = 0;                                       //@@@          def tot synchroney L1        ---@
        double R_syncrony_layer1 = 0;                                       //@@@          def tot synchroney L1        ---@

        
        
        
        //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        //@@@                                        time loop                  //@@@                                       ---@  @
        //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                       ---@  @
        ofstream time_syncroney("Save/Syncrony(time_SyncL1_SyncL2)/k="+         //@@@                                       ---@  @
                                strcoupling+".txt");                            //@@@                                       ---@  @
        double time_step = double(data[2]);                                     //@@@     reset time for new time           ---@  @        
        for (time_step;time_step < int(data[4]/data[3]);time_step++){           //@@@                                       ---@  @
            double time_loop=time_step*data[3];                                 //@@@                                       ---@  @
            Runge_Kutta_4(Number_of_node,                                       //@@@   Runge-Kutta 4th Order Method  L1    ---@  @
                        data[3],                                                //@@@                                       ---@  @
                        coupling,                                               //@@@                                       ---@  @
                        frequency_layer1,                                       //@@@                                       ---@  @
                        inter_layer1,                                           //@@@                                       ---@  @
                        bdj_layer1,                                             //@@@                                       ---@  @
                        frust_layer1,                                           //@@@                                       ---@  @
                        Intrafrust_layer1,                                      //@@@                                       ---@  @
                        adj_layer1,                                             //@@@                                       ---@  @
                        Phases_layer1_previous,                                 //@@@                                       ---@  @
                        Phases_layer2_previous,                                 //@@@                                       ---@  @
                        Phases_next_layer1);                                    //@@@                                       ---@  @
            Runge_Kutta_4(Number_of_node,                                       //@@@   Runge-Kutta 4th Order Method  L2    ---@  @
                        data[3],                                                //@@@                                       ---@  @
                        coupling,                                               //@@@                                       ---@  @
                        frequency_layer2,                                       //@@@                                       ---@  @
                        inter_layer2,                                           //@@@                                       ---@  @
                        bdj_layer2,                                             //@@@                                       ---@  @
                        frust_layer2,                                           //@@@                                       ---@  @
                        Intrafrust_layer2,                                      //@@@                                       ---@  @
                        adj_layer2,                                             //@@@                                       ---@  @
                        Phases_layer2_previous,                                 //@@@                                       ---@  @
                        Phases_layer1_previous,                                 //@@@                                       ---@  @
                        Phases_next_layer2);                                    //@@@                                       ---@  @
            Phases_layer1_previous = for_loop_equal(Phases_next_layer1);        //@@@           Back to the future L1       ---@  @
            Phases_layer2_previous = for_loop_equal(Phases_next_layer2);        //@@@           Back to the future L2       ---@  @
            check_scale(Number_of_node,Phases_layer1_previous);                 //@@@       scale phases in -pi tp pi L1    ---@  @
            check_scale(Number_of_node,Phases_layer2_previous);                 //@@@       scale phases in -pi tp pi L2    ---@  @
            double syncrony_layer1 = order_parameter(Number_of_node,            //@@@                                       ---@  @
                                    Phases_layer1_previous);                    //@@@     order parameters (Synchroney) L1  ---@  @
            double syncrony_layer2 = order_parameter(Number_of_node,            //@@@                                       ---@  @
                                    Phases_layer2_previous);                    //@@@     order parameters (Synchroney) L2  ---@  @


            double Tsyncrony_layer1 = TGroup_order_parameter(Phases_layer1_previous); 
            double Lsyncrony_layer1 = LGroup_order_parameter(Phases_layer1_previous); 
            double Msyncrony_layer1 = MGroup_order_parameter(Phases_layer1_previous); 
            double Rsyncrony_layer1 = RGroup_order_parameter(Phases_layer1_previous); 

            //------------------------------------------------------------------//@@@------------------------------------------@  @
            //Phases_layer2 << time_loop << '\\t';                                 //@@@                                       ---@  @
            //Phases_layer1 << time_loop << '\\t';                                 //@@@                                       ---@  @
            //for (int i = 0; i < Number_of_node; i++){                           //@@@                                       ---@  @
            //    Phases_layer2 << std::fixed << std::setprecision(2) <<          //@@@                                       ---@  @
            //                                Phases_layer2_previous[i] << '\\t';  //@@@--->  print phase data in file .txt    ---@  @
            //    Phases_layer1 << std::fixed << std::setprecision(2) <<          //@@@                                       ---@  @
            //                                Phases_layer1_previous[i] << '\\t';  //@@@                                       ---@  @
            //}                                                                   //@@@                                       ---@  @
            //Phases_layer2 << endl;                                              //@@@                                       ---@  @
            //Phases_layer1 << endl;                                              //@@@                                       ---@  @
            //------------------------------------------------------------------//@@@------------------------------------------@  @
            if (time_step>=Number_Steps_time_stationary){                       //@@@                                       ---@  @
                Total_syncrony_layer1 += syncrony_layer1;                       //@@@         calculate synchroney L1       ---@  @
                Total_syncrony_layer2 += syncrony_layer2;                       //@@@         calculate synchroney L2       ---@  @
                T_syncrony_layer1 += Tsyncrony_layer1;                       //@@@         calculate synchroney L1       ---@  @
                L_syncrony_layer1 += Lsyncrony_layer1;                       //@@@         calculate synchroney L1       ---@  @
                M_syncrony_layer1 += Msyncrony_layer1;                       //@@@         calculate synchroney L1       ---@  @
                R_syncrony_layer1 += Rsyncrony_layer1;                       //@@@         calculate synchroney L1       ---@  @




            }                                                                   //@@@                                       ---@  @
            time_syncroney<<time_loop<< '\\t'<<syncrony_layer1<< '\\t'            //@@@   print synchrony data in file .txt   ---@  @
                          <<syncrony_layer2<<endl;                              //@@@                                       ---@  @
            //cout<<time_loop<< '\\t'<<syncrony_layer1<< '\\t'                    //@@@           cout synchrony data         ---@  @
                          //<<syncrony_layer2<<endl;                            //@@@                                       ---@  @
        }                                                                       //@@@                                       ---@  @
        //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        Total_syncrony_layer1=Total_syncrony_layer1/                            //@@@                                       ---@
                              (Number_Steps_time_stationary*4);                 //@@@    calculate total sync and pint it   ---@
        Total_syncrony_layer2=Total_syncrony_layer2/                            //@@@                                       ---@
                              (Number_Steps_time_stationary*4);                 //@@@                                       ---@
        time_t end = time(NULL);                                                //@@@             time of run               ---@
        cout<<"k=" <<strcoupling << '\\t' <<"r="<< Total_syncrony_layer2 <<      //@@@       cout total synchrony data       ---@
        '\\t' <<"Ex Time: "<< (double)(end-start)<<" Sec"<<endl;                 //@@@                                       ---@
        Average_Syncrony << strcoupling << '\\t' << Total_syncrony_layer1 <<     //@@@          print total synchrony        ---@
                         '\\t' << Total_syncrony_layer2<< '\\t' <<                //@@@            data in file .txt          ---@
                         (double)(end-start) << endl;                           //@@@                                       ---@


        T_syncrony_layer1=T_syncrony_layer1/(Number_Steps_time_stationary*4);
        L_syncrony_layer1=L_syncrony_layer1/(Number_Steps_time_stationary*4);
        M_syncrony_layer1=M_syncrony_layer1/(Number_Steps_time_stationary*4);
        R_syncrony_layer1=R_syncrony_layer1/(Number_Steps_time_stationary*4);

        Group_Syncrony<< strcoupling << '\\t' << T_syncrony_layer1 << '\\t'<<L_syncrony_layer1 << '\\t'<<M_syncrony_layer1 <<'\\t'<<R_syncrony_layer1 <<endl;                           //@@@                                       ---@

        //Phases_layer2.close();                                                  //@@@                                       ---@
        //Phases_layer1.close();                                                  //@@@                                       ---@
    }                                                                           //@@@                                       ---@
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    ofstream Last_Phase_layer1("Save/Last_Phase/layer1.txt");                   //@@@                                       ---
    ofstream Last_Phase_layer2("Save/Last_Phase/layer2.txt");                   //@@@                                       ---
    for (int i = 0; i < Number_of_node; i++){                                   //@@@                                       ---
        Last_Phase_layer1 << Phases_layer1_previous[i] << endl;                 //@@@--->   print last coupling phases      ---
        Last_Phase_layer2 << Phases_layer2_previous[i] << endl;                 //@@@                                       ---
    }                                                                           //@@@                                       ---
    //--------------------------------------------------------------------------//@@@------------------------------------------
    Average_Syncrony.close();                                                   //@@@                                       ---
    Last_Phase_layer1.close();                                                  //@@@                                       ---
    Last_Phase_layer2.close();                                                  //@@@                                       ---
    delete Phases_layer1_previous;                                              //@@@                                       ---
    delete Phases_layer2_previous;                                              //@@@                                       ---
    delete Phases_next_layer1;                                                  //@@@                                       ---
    delete Phases_next_layer2;                                                  //@@@                                       ---
    //@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                       ---                                                                             //@@@@                                   ---
    return 0;                                                                   //@@@     dont return any thing             ---
}                                                                               //@@@                                       ---
//@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@---
//-----------------------------------------------------------------------------------------------------------------------------


"""
# ذخیره در فایل
with open('./main.cpp', 'w') as f:
    f.write(cpp_code)


print("✅  .با موفقیت ساخته شد main.cpp فایل ")

✅  .با موفقیت ساخته شد main.cpp فایل 


In [4]:
import subprocess
import os

# مسیر فایل‌ها
folder = './'
cpp_file = os.path.join(folder, 'main.cpp')
exe_file = os.path.join(folder, 'main')  # فایل اجرایی هم توی همون پوشه ذخیره بشه

# 1. کامپایل با g++
compile_result = subprocess.run(['g++', cpp_file, '-o', exe_file])

# 2. بررسی موفقیت کامپایل
if compile_result.returncode == 0:
    print("✅ کامپایل موفق بود. حالا برنامه در انتظار اجرا میباشد")
else:
    print("❌ خطا در کامپایل")


✅ کامپایل موفق بود. حالا برنامه در انتظار اجرا میباشد


In [5]:
import subprocess
import os

# مسیر فایل اجرایی
exe_path = './main.exe'

# بررسی اینکه فایل وجود داره یا نه (اختیاری ولی خوبه)
if os.path.exists(exe_path):
    print("✅ فایل پیدا شد، در حال اجرا...\n")

    # اجرای فایل و دریافت خروجی
    run_result = subprocess.run([exe_path], capture_output=True, text=True)

    # چاپ خروجی
    print(run_result.stdout)
else:
    print("❌ فایل main.exe پیدا نشد.")


✅ فایل پیدا شد، در حال اجرا...


File had 7 rows
data[1]=	1000
data[2]=	0
data[3]=	0.1
data[4]=	4
data[5]=	1.5
data[6]=	0.01
data[7]=	1.51
|------------------------------------------------------|

W of Layer1	loaded	 First data=-0.5	Lastst data=0.5
W of Layer2	loaded	 First data=0.5	Lastst data=-0.5
I of origin1	loaded	 First data=0.567	Lastst data=-1.568
I of origin2	loaded	 First data=0.789	Lastst data=1.943
A of Layer1	loaded	 First data=1	Lastst data=1
A of Layer2	loaded	 First data=1	Lastst data=1
B of Layer1to2	loaded	 First data=1	Lastst data=1
B of Layer2to1	loaded	 First data=1	Lastst data=1
a of Layer1to2	loaded	 First data=1.57	Lastst data=1.57
a of Layer2to1	loaded	 First data=1.57	Lastst data=1.57
b of Layer1	loaded	 First data=0	Lastst data=0
b of Layer2	loaded	 First data=0	Lastst data=0
L of Layer1to2	loaded	 First data=10	Lastst data=10
L of Layer2to1	loaded	 First data=10	Lastst data=10
|------------------------------------------------------|

k=1.50	r=inf	Ex Time: 6 